## The Shortest Path Problem

Another special case of minimal-cost network-flow problem.

$$\text{min} \sum_{(i, j)\in{A}} c_{ij} x_{ij}$$

$\text{subject to}$

\begin{equation}
  \sum_{(i, j)\in{A}} x_{ij} - \sum_{(j, i)\in{A}} x_{ji}=\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if}\ i \text{ is the origin node} \\
    -1, & \text{if}\ i \text{ is the destination node} \\
    0, & \text{otherwise}
  \end{array}\right.
  \quad \forall{i} \in{N}
\end{equation}

$$x_{ij} \in \{0, 1\} \quad \forall{(i, j)} \in{A}$$

Can be solved using LP but Dijkstra's algorithm is more efficient.

In [2]:
network_data = readcsv("simple_network.csv", header=true)
data = network_data[1]
header = network_data[2]

start_nodes = round.(Int64, data[:, 1])
end_nodes = round.(Int64, data[:, 2])
c = data[:, 3] # cost

origin = 1 # origin node
destination = 5 # destination node

num_nodes = max(maximum(start_nodes), maximum(end_nodes))
num_links = length(start_nodes)

8

In [3]:
using LightGraphs

graph = Graph(num_nodes)
distance_matrix = Inf * ones(num_nodes, num_nodes)

for i = 1:num_links
    add_edge!(graph, start_nodes[i], end_nodes[i])
    distance_matrix[start_nodes[i], end_nodes[i]] = c[i]
end

In [4]:
# Dijkstra's algorithm is a one-to-all algorithm,
# i.e. finds shortest paths from a single origin to all nodes
state = dijkstra_shortest_paths(graph, origin, distance_matrix)

LightGraphs.DijkstraState{Float64,Int64}([0, 1, 1, 3, 3], [0.0, 2.0, 5.0, 6.0, 7.0], Array{Int64,1}[Int64[], Int64[], Int64[], Int64[], Int64[]], [1, 1, 2, 2, 2], Int64[])

In [5]:
# Extracts binary x vector
function get_shortest_x(path, start_nodes, end_nodes, origin, destination)
    x = zeros(Int, length(start_nodes))
    
    for i = 1:length(path)-1
        start = path[i]
        end_ = path[i+1]
        
        for j = 1:length(start_nodes)
            if start_nodes[j] == start && end_nodes[j] == end_
                x[j] = 1
                break
            end
        end
    end
    
    return x
end

# Returns indices of the nodes in the shortest path
path = enumerate_paths(state, destination)
println("The shortest path is: ", path)

x = get_shortest_x(path, start_nodes, end_nodes, origin, destination)
println("x vector: ", x)

println("Cost is $(state.dists[destination])")
println("Cost is $(dot(c, x))")
println("Cost is $(c' * x)")

The shortest path is: [1, 3, 5]
x vector: [0, 1, 0, 0, 1, 0, 0, 0]
Cost is 7.0
Cost is 7.0
Cost is 7.0


In [6]:
[start_nodes end_nodes x]

8×3 Array{Int64,2}:
 1  2  0
 1  3  1
 2  3  0
 3  4  0
 3  5  1
 4  1  0
 4  5  0
 5  2  0